# Libraries

In [1]:
import os
import socket
from urllib.request import urlretrieve

import pandas as pd
from tqdm.notebook import tqdm

# Download

In [2]:
dataset = 'https://raw.githubusercontent.com/rubenros1795/iconicity/master/data/dataset/TimesSquareKiss-sift-corrected.tsv'
sample_size = 600

socket.setdefaulttimeout(10)

df = pd.read_csv(dataset, sep = '\t')
sample = df.sample(sample_size, random_state = 42)

if not os.path.exists('images'):
    os.mkdir('images')

for row in tqdm(sample.itertuples(), total=sample_size):
    if row.image_url_full != 'na':
        url = row.image_url_full
    else:
        url = row.image_url_partial
        
    file_extension = url[url.rfind('.'):]
    if '?' in file_extension:
        file_extentsion = file_extension[:file_extension.find('?')]
    try:
        urlretrieve(url, "images/{}{}".format(row.Index, file_extension))
    except:
        continue

# Adjust files
### List of files

In [21]:
files = os.listdir("images")

### Extract number (to sort) and extension (need adjustments)

In [60]:
numbers = []
extensions = []
for file in files:
    number, extension = file.split(".")
    numbers.append(int(number))
    extensions.append(extension)

In [64]:
labeled_by = ["All"]*44+100*["Joey"]+100*["Asher"]+100*["Ioannis"]+100*["Evangelos"]

### Make DF

In [67]:
df = pd.DataFrame()
df["file"] = files
df["number"] = numbers
df["extension"] = extensions

In [68]:
df.head()

,file,number,extension
0,56374.jpg,56374,jpg
1,9055.jpg,9055,jpg
2,51948.jpeg,51948,jpeg
3,11771.jpg,11771,jpg
4,20235.jpg,20235,jpg


### Sort

In [73]:
df_sorted = df.sort_values("number").reset_index(drop=True)
df_sorted["labeled_by"] = labeled_by
# df_sorted.head()

### Loop though extensions and find solitions

In [91]:
numbers = df_sorted["number"]
extensions = df_sorted["extension"]
adjusted = []
for ext in df_sorted["extension"]:
    if ext in ["jpg", "jpeg", "png"]:
        adjusted.append(ext)
    elif ext.startswith("jpg"):
        adjusted.append("jpg")
    elif ext.startswith("jpeg"):
        adjusted.append("jpeg")
    elif ext.startswith("png"):
        adjusted.append("png")
    else:
        print(ext)
        adjusted.append(ext)
df_sorted["adjusted_extension"] = adjusted

8
gif
webp?maxwidth=728&fidelity=grand
php?table=ans&No=595959&data=front&date=2009-04-16
gif
gif
net%2Fcfile%2Fblog%2F23447236533BF61909
webp


### New filenames

In [104]:
numbers = df_sorted["number"]
adjusted_extensions = df_sorted["adjusted_extension"]
new_filenames = []
for i in range(len(df_sorted)):
    new_filename = str(numbers[i]) + "." + adjusted_extensions[i]
    new_filenames.append(new_filename)
df_sorted["new_filename"] = new_filenames

In [105]:
df_sorted.head()

,file,number,extension,labeled_by,adjusted_extension,new_filename
0,144.jpg,144,jpg,All,jpg,144.jpg
1,212.jpg,212,jpg,All,jpg,212.jpg
2,251.jpg?resize=450%2C673&ssl=1,251,jpg?resize=450%2C673&ssl=1,All,jpg,251.jpg
3,253.jpg?w=1100,253,jpg?w=1100,All,jpg,253.jpg
4,273.jpg,273,jpg,All,jpg,273.jpg


## Rename

In [108]:
for i in range(len(df_sorted)):
    old = df_sorted["file"][i]
    new = df_sorted["new_filename"][i]
    if old != new:
        os.rename(f"images/{old}", f"images/{new}")
#returns error if run again

FileNotFoundError: [Errno 2] No such file or directory: 'images/251.jpg?resize=450%2C673&ssl=1' -> 'images/251.jpg'

In [111]:
df_new = df_sorted[["new_filename", "labeled_by"]]

In [112]:
Nones = [None]*444

In [113]:
df_new["Joey"] = Nones
df_new["Asher"] = Nones
df_new["Ioannis"] = Nones
df_new["Evangelos"] = Nones

<ipython-input-113-e1e12f57cef1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["Joey"] = Nones
<ipython-input-113-e1e12f57cef1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["Asher"] = Nones
<ipython-input-113-e1e12f57cef1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [116]:
df_new.head()

,new_filename,labeled_by,Joey,Asher,Ioannis,Evangelos
0,144.jpg,All,None,None,None,None
1,212.jpg,All,None,None,None,None
2,251.jpg,All,None,None,None,None
3,253.jpg,All,None,None,None,None
4,273.jpg,All,None,None,None,None


In [117]:
df_new.to_csv("exercise4_1.csv")